In [18]:
from synthrad_conversion.utils.my_configs_yacs import init_cfg
from dataprocesser.step1_init_data_list import init_dataset
from tqdm import tqdm
config_path = 'tutorial_config.yaml'
opt=init_cfg(config_path)
model_name_path='Infer_'+opt.model_name + opt.name_prefix
print(model_name_path)
dataset_name = 'multimodal_csv'
loader, opt, my_paths = init_dataset(opt, model_name_path, dataset_name)
train_ds = loader.train_ds
val_ds  = loader.val_ds
print(len(train_ds))
print(len(val_ds))


Infer_ddpm2d_seg2med
create combined segmentation dataset with assigned value
create base dataset
combined segmentation assigned dataset use keys: ['source', 'target', 'mask']
use train csv: tutorial_train.csv
use test csv: tutorial_test.csv
use keys for creating volume dataset:  ['source', 'target', 'mask']
model name:  ddpm2d_seg2med
val_use_patch:  False
527
135


In [19]:
print(len(train_ds))
for item in train_ds[0]:
    print(f'{item}: {train_ds[0][item]}')

527
source: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct_seg.nii.gz
target: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct.nii.gz
seg_tissue: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct_seg_tissue.nii.gz
mask: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct.nii.gz
modality: 0
A_paths: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct.nii.gz
B_paths: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct_seg.nii.gz
mask_path: E:\Projects\yang_proj\data\synthrad\Task1\pelvis\1PA004\ct_seg.nii.gz
Aorta_diss: 0
patient_ID: 1PA004


# process

- add_CreateContour_MergeSegTissue_MergeMask_MaskHUAssign_transforms
    - CreateBodyContourTransformd
    - MergeSegTissueTransformd
    - MergeMasksTransformd
    - MaskHUAssigmentd

- add_Windowing_ZeroShift_ContourFilter_A_B_transforms
    - ThresholdIntensityd
    - ThresholdIntensityd
    - ShiftIntensityd
    - UseContourToFilterImaged

- add_normalization_multimodal
    - NormalizationMultimodal

In [20]:
import os 
import shutil
from dataprocesser.preprocess_MR.step3_vibe_resetsignal import calculate_signal_GRE_T1, calculate_signal_T2_SPACE, calculate_signal_vibe, calculate_signal_vibe_opp, calculate_signal_vibe_dixon
from dataprocesser.simulation_functions import _create_body_contour, _merge_seg_tissue, _merge_seg_contour, _assign_value_ct, _assign_value_mr
import csv
import nibabel as nib
def create_csv_modality_info_file(dataset_list, output_mr_csv_file):
    with open(output_mr_csv_file, 'w', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(['id', 'Aorta_diss', 'prior' ,'img', 'modality'])
        csvwriter.writerows(dataset_list) 

def zip_to_csv_list(images, priors, patient_IDs, ad_list, modality_list):  
    datalist = [[id,ad,prior,image,modality] 
                            for id,ad,prior,image, modality in zip(patient_IDs, ad_list, priors, images, modality_list)]
    return datalist

def output_dataset(item, 
                   output_folder = r'E:\Projects\yang_proj\data\seg2med\seg2med_train', 
                   csv_file_ct = r'synthrad_conversion\TA2_CT.csv',
                   csv_file_mr = r'synthrad_conversion\TA2_T1T2.csv'):
    
    
    source = item['source']
    target = item['target']
    seg_tissue = item['seg_tissue']
    mask = item['mask']
    modality = item['modality']
    Aorta_diss = item['Aorta_diss']
    patient_ID = item['patient_ID']

    output_dir = os.path.join(output_folder,patient_ID)
    os.makedirs(output_dir,exist_ok=True)

    target_data_df = nib.load(target)
    seg_data_df = nib.load(source)
    seg_tissue_data_df = nib.load(seg_tissue)

    target_data = target_data_df.get_fdata()
    target_affine = target_data_df.affine
    seg_data = seg_data_df.get_fdata()
    seg_tissue_data = seg_tissue_data_df.get_fdata()


    modality_config = {
        0: {
            'body_threshold': -500,
            'assign_fn': _assign_value_ct,
            'csv_file': csv_file_ct,
            'signal_fn': None
        },
        1: { # t1 weighted gradient echo
            'body_threshold': 100,
            'assign_fn': _assign_value_mr,
            'csv_file': csv_file_mr,
            'signal_fn': calculate_signal_GRE_T1
        },
        2: { # t2 space
            'body_threshold': 10,
            'assign_fn': _assign_value_mr,
            'csv_file': csv_file_mr,
            'signal_fn': calculate_signal_T2_SPACE
        },
        3: { # vibe in
            'body_threshold': 30,
            'assign_fn': _assign_value_mr,
            'csv_file': csv_file_mr,
            'signal_fn': calculate_signal_vibe
        },
        4: { # vibe opp
            'body_threshold': 30,
            'assign_fn': _assign_value_mr,
            'csv_file': csv_file_mr,
            'signal_fn': calculate_signal_vibe_opp
        },
        5: { # vibe dixon
            'body_threshold': 30,
            'assign_fn': _assign_value_mr,
            'csv_file': csv_file_mr,
            'signal_fn': calculate_signal_vibe_dixon
        }
    }

    # 获取当前模态参数
    config = modality_config.get(modality)
    if config is None:
        raise ValueError(f"Unsupported modality: {modality}")

    # 通用处理流程
    contour = _create_body_contour(target_data, config['body_threshold'], body_mask_value=1)
    merged_seg_tissue = _merge_seg_tissue(seg_data, seg_tissue_data)
    merged_seg_tissue_contour = _merge_seg_contour(merged_seg_tissue, contour)

    # 指定 assign 函数处理 CT 或 MR
    if config['signal_fn'] is not None:
        prior_image = config['assign_fn'](config['csv_file'], merged_seg_tissue_contour, config['signal_fn'])
    else:
        prior_image = config['assign_fn'](config['csv_file'], merged_seg_tissue_contour)

    processed_img = nib.Nifti1Image(prior_image, target_affine)
    contour_img = nib.Nifti1Image(contour, target_affine)

    target_output_name = os.path.basename(target)
    target_output_path = os.path.join(output_dir, target_output_name)

    seg_output_name = os.path.basename(source)
    seg_output_path = os.path.join(output_dir, seg_output_name)

    seg_tissue_output_name = os.path.basename(seg_tissue)
    seg_tissue_output_path = os.path.join(output_dir, seg_tissue_output_name)

    prior_image_output_name = os.path.basename(target).replace('.nii', '_prior_image.nii')
    prior_image_output_path = os.path.join(output_dir, prior_image_output_name)

    contour_output_name = os.path.basename(target).replace('.nii', '_contour.nii')
    contour_output_path = os.path.join(output_dir, contour_output_name)

    '''shutil.copy(target, target_output_path)
    shutil.copy(source, seg_output_path)
    shutil.copy(seg_tissue, seg_tissue_output_path)
    nib.save(processed_img, prior_image_output_path)'''
    nib.save(contour_img, contour_output_path)

    return patient_ID,Aorta_diss,prior_image_output_path,target_output_path, modality

In [21]:
item = train_ds[0] # test synthrad ct
item = train_ds[150] # test synthrad mr T1 GRE
#item = train_ds[155] # test synthrad mr T1 GRE
#item = train_ds[160] # test synthrad mr T1 GRE
#item = train_ds[165] # test synthrad mr T1 GRE
#item = train_ds[170] # test synthrad mr T1 GRE
#item = train_ds[240] # test synthrad mr T2 SPACE
#item = train_ds[245] # test synthrad mr T2 SPACE
#item = train_ds[250] # test synthrad mr T2 SPACE
item = train_ds[255] # test synthrad mr T2 SPACE
#item = train_ds[344] # test vibe_in
#item = train_ds[388] # test vibe_opp
#item = train_ds[395] # test vibe_dixon
output_dataset(item, output_folder = r'E:\Projects\yang_proj\data\seg2med\seg2med_train')

min, max of combine: 0 204
min, max of assignment: 0 28


('1PC004',
 0,
 'E:\\Projects\\yang_proj\\data\\seg2med\\seg2med_train\\1PC004\\mr_prior_image.nii.gz',
 'E:\\Projects\\yang_proj\\data\\seg2med\\seg2med_train\\1PC004\\mr.nii.gz',
 2)

In [22]:
images = []
priors = []
patient_IDs = []
ad_list = []
modality_list = []
output_test_csv = 'tutorial_test_prior.csv'
for item in tqdm(val_ds):
    patient_ID,Aorta_diss,prior_image_output_path,target_output_path, modality = output_dataset(item, output_folder = r'E:\Projects\yang_proj\data\seg2med\seg2med_test')
    images.append(target_output_path)                # 原始图像路径
    priors.append(prior_image_output_path)           # prior 图像路径
    patient_IDs.append(patient_ID)
    ad_list.append(Aorta_diss)
    modality_list.append(modality)
#dataset_list = zip_to_csv_list(images, priors, patient_IDs, ad_list, modality_list)
#create_csv_modality_info_file(dataset_list, output_test_csv)


images = []
priors = []
patient_IDs = []
ad_list = []
modality_list = []
output_train_csv = 'tutorial_train_prior.csv' 
for item in tqdm(train_ds):
    patient_ID,Aorta_diss,prior_image_output_path,target_output_path, modality = output_dataset(item, output_folder = r'E:\Projects\yang_proj\data\seg2med\seg2med_train')
    images.append(target_output_path)                # 原始图像路径
    priors.append(prior_image_output_path)           # prior 图像路径
    patient_IDs.append(patient_ID)
    ad_list.append(Aorta_diss)
    modality_list.append(modality)
#dataset_list = zip_to_csv_list(images, priors, patient_IDs, ad_list, modality_list)
#create_csv_modality_info_file(dataset_list, output_train_csv)



 27%|██▋       | 36/135 [01:43<04:49,  2.92s/it]

min, max of combine: 0 204
min, max of assignment: 0 9


 27%|██▋       | 37/135 [01:45<04:46,  2.92s/it]

min, max of combine: 0 204
min, max of assignment: 0 9


 28%|██▊       | 38/135 [01:48<04:30,  2.79s/it]

...

 75%|███████▍  | 395/527 [20:36<07:38,  3.47s/it]

min, max of combine: 0 204
min, max of assignment: 0 6


 75%|███████▌  | 396/527 [20:42<08:49,  4.04s/it]

min, max of combine: 0 204
min, max of assignment: 0 6


100%|██████████| 527/527 [34:51<00:00,  3.97s/it]


## test signal values for subcutaneous_fat, torso_fat, skeletal_muscle

In [5]:
W = 0.9
F = 0.1
delta_omega=220.0
TE = 2.3
phase_shift = np.exp(1j * 2 * np.pi * delta_omega * TE / 1000)  # 注意TE
print(F*phase_shift)
print(np.abs(W + F * phase_shift))



(-0.09992894726405893-0.0037690182669934736j)
0.8000799303348972


In [12]:
# subcutaneous_fat, torso_fat, skeletal_muscle
print(calculate_signal_vibe_dixon(275,90,95))
print(calculate_signal_vibe_dixon(300,80,90))
print(calculate_signal_vibe_dixon(1142,45,80))

6.76900410503465
6.126166793924495
2.2167670709685465


In [13]:
print(calculate_signal_T2_SPACE(275,90,95))
print(calculate_signal_T2_SPACE(300,80,90))
print(calculate_signal_T2_SPACE(1142,45,80))

17.930725823677232
13.784382191746642
2.358633605612473


# check the data distribution, preparing for further normalization into training

In [2]:
from synthrad_conversion.utils.my_configs_yacs import init_cfg
from dataprocesser.step1_init_data_list import init_dataset
config_path = 'tutorial2_config_prior.yaml'
opt=init_cfg(config_path)
model_name_path='Infer_'+opt.model_name + opt.name_prefix
print(model_name_path)
dataset_name = 'multimodal_prior_csv'
loader, opt, my_paths = init_dataset(opt, model_name_path, dataset_name)

train_ds = loader.train_ds
val_ds  = loader.val_ds
print(len(train_ds))
print(len(val_ds))

C:\Users\zy7\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Infer_ddpm2d_seg2med
create combined segmentation dataset with assigned value
create base dataset
combined segmentation assigned dataset use keys: ['source', 'target', 'mask']
use train csv: tutorial2_train_prior.csv
use test csv: tutorial2_test_prior.csv
use keys for creating volume dataset:  ['source', 'target', 'mask']
model name:  ddpm2d_seg2med
val_use_patch:  False
527
135


In [17]:
import napari
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
def data_distribution(item, do_boxplot=False, hist_bins=200):
    source = item['source']
    target = item['target']
    modality = item['modality']
    patient_ID = item['patient_ID']


    target_data_df = nib.load(target)
    prior_data_df = nib.load(source)

    target_data = target_data_df.get_fdata()
    target_affine = target_data_df.affine
    prior_data = prior_data_df.get_fdata()

    print("===> Patient:", patient_ID, "| Modality:", modality)
    print("Target image - min:", np.min(target_data), "max:", np.max(target_data))
    print("Prior image  - min:", np.min(prior_data), "max:", np.max(prior_data))

    # 去掉无效区域（如为 0 或负值的区域）
    target_valid = target_data[target_data > 0]
    prior_valid = prior_data[prior_data > 0]

    # 分位点
    t10, t90 = np.percentile(target_valid, [10, 90])
    p10, p90 = np.percentile(prior_valid, [10, 90])

    t5, t95 = np.percentile(target_valid, [5, 95])
    print(f"Target intensity 10%: {t10:.2f}, 90%: {t90:.2f}")
    print(f"Target intensity 5%: {t5:.2f}, 95%: {t95:.2f} \n")

    if do_boxplot:
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 2, 1)
        #plt.hist(target_data.flatten(), bins=hist_bins, color='steelblue', alpha=0.8)
        plt.boxplot(target_data.flatten())
        plt.title("Image Pixel Value Distribution")
        plt.subplot(1, 2, 2)
        #plt.hist(prior_data.flatten(), bins=hist_bins, color='steelblue', alpha=0.8) 
        plt.boxplot(prior_data.flatten()) 
        plt.title("Prior Pixel Value Distribution")

        plt.tight_layout()
        plt.show()

    '''viewer = napari.Viewer()
    viewer.add_image(target_data, name='Target', colormap='gray')
    viewer.add_image(prior_data, name='Prior', colormap='magma', blending='additive')
    napari.run()'''


# 收集所有 item
items_to_check = [
    train_ds[0],    # synthrad CT
    train_ds[150],  # synthrad MR T1 GRE
    train_ds[155],
    train_ds[160],
    train_ds[165],
    train_ds[240],  # MR T2 SPACE
    train_ds[242],
    train_ds[245],
    train_ds[247],
    train_ds[255],
    train_ds[257],
    train_ds[340],  # vibe_in
    train_ds[342],
    train_ds[345],
    train_ds[347],
    train_ds[350],
    train_ds[360],  # vibe_opp
    train_ds[362],
    train_ds[365],
    train_ds[367],
    train_ds[370],
    train_ds[372],
    train_ds[392],  # vibe_dixon
    train_ds[393],
    train_ds[394],
    train_ds[395],
    train_ds[396],
    train_ds[397],
]

# 依次输出每个 item 的直方图及分位点
for item in items_to_check:
    data_distribution(item, hist_bins=200)

===> Patient: 1PA004 | Modality: 0
Target image - min: -1024.0 max: 1458.0
Prior image  - min: -1000.0 max: 700.0
Target intensity 10%: 22.00, 90%: 216.00
Target intensity 5%: 12.00, 95%: 346.00 

===> Patient: 1PA147 | Modality: 1
Target image - min: 0.0 max: 545.3027954101562
Prior image  - min: 0.0 max: 9.0
Target intensity 10%: 0.77, 90%: 273.68
Target intensity 5%: 0.28, 95%: 287.75 

===> Patient: 1PA113 | Modality: 1
Target image - min: 0.0 max: 629.5413208007812
Prior image  - min: 0.0 max: 9.0
Target intensity 10%: 0.67, 90%: 319.08
Target intensity 5%: 0.24, 95%: 338.75 

===> Patient: 1PA094 | Modality: 1
Target image - min: 0.0 max: 2637.25537109375
Prior image  - min: 0.0 max: 9.0
Target intensity 10%: 18.24, 90%: 1206.17
Target intensity 5%: 6.43, 95%: 1400.43 

===> Patient: 1PA022 | Modality: 1
Target image - min: 0.0 max: 668.1495361328125
Prior image  - min: 0.0 max: 9.0
Target intensity 10%: 0.62, 90%: 373.21
Target intensity 5%: 0.25, 95%: 394.19 

===> Patient: 1PC

In [ ]:
# visualize a slice:
import matplotlib.pyplot as plt
def plottorchdata(data, i, slice_id):
    data = data.cpu().numpy().squeeze().squeeze()
    dataimg = f'tutorial_{i}.png'
    plt.figure(i)
    plt.imshow(data[..., slice_id], cmap='gray')
    plt.savefig(dataimg)
slice_id = 50
plottorchdata(first_input, 0, slice_id)
plottorchdata(first_target, 1, slice_id)

In [ ]:
import synthrad_conversion.networks as networks
from synthrad_conversion.networks.model_registry import MODEL_REGISTRY
for model in MODEL_REGISTRY:
    print(model)

c:\Users\zy7\.conda\envs\torch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zy7\.conda\envs\torch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zy7\.conda\envs\torch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\zy7\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ddpm2d_seg2med
spade_ddpm2d_seg2med
pix2pix
cycle_gan
AttentionUnet
resUnet
